In [0]:
WITH state_data AS (
    SELECT 
        state_gdp.state_code,
        state_gdp.business_year,
        state_gdp.average_gdp AS avg_gdp_state,
        usa_gdp.average_gdp AS avg_gdp_usa
    FROM dev_catalog.gold.state_gdp_dim AS state_gdp
    JOIN dev_catalog.gold.state_gdp_dim AS usa_gdp
        ON state_gdp.business_year = usa_gdp.business_year
        AND usa_gdp.state_code = 'USA'
    WHERE state_gdp.state_code != 'USA'
),
exploded_rates AS (
    SELECT 
        plan_id,
        state_code,
        age_category,
        age_rate,
        struct_col.business_year
    FROM dev_catalog.gold.rates_fact
    LATERAL VIEW EXPLODE(age_category_and_year) exploded AS struct_col
    LATERAL VIEW EXPLODE(
        MAP(
            'age_0_19', struct_col.age_0_19,
            'age_20_29', struct_col.age_20_29,
            'age_30_39', struct_col.age_30_39,
            'age_40_49', struct_col.age_40_49,
            'age_50_59', struct_col.age_50_59,
            'age_60_plus', struct_col.age_60_plus
        )
    ) exploded_map AS age_category, age_rate
),
state_rates_agg AS (
    SELECT
        state_code,
        business_year,
        AVG(age_rate) AS avg_rate
    FROM exploded_rates
    GROUP BY state_code, business_year
),
national_rates_agg AS (
    SELECT
        business_year,
        AVG(age_rate) AS avg_rate_national
    FROM exploded_rates
    GROUP BY business_year
),
rates_agg AS (
    SELECT 
        r.state_code,
        r.business_year,
        r.avg_rate,
        n.avg_rate_national
    FROM state_rates_agg r
    JOIN national_rates_agg n
        ON r.business_year = n.business_year
),
fairness_index_calc AS (
    SELECT 
        r.state_code,
        r.business_year,
        (r.avg_rate/s.avg_gdp_state)/(r.avg_rate_national/s.avg_gdp_usa) AS fairness_index
    FROM rates_agg r
    JOIN state_data s 
        ON r.state_code = s.state_code 
        AND r.business_year = s.business_year
)
SELECT 
    state_code,
    business_year,
    fairness_index
FROM (
    SELECT 
        state_code,
        business_year,
        fairness_index,
        ROW_NUMBER() OVER (PARTITION BY business_year ORDER BY fairness_index DESC) AS rank_desc,
        ROW_NUMBER() OVER (PARTITION BY business_year ORDER BY fairness_index ASC) AS rank_asc
    FROM fairness_index_calc
) ranked
WHERE rank_desc <= 10 OR rank_asc <= 10
ORDER BY business_year, fairness_index;

state_code,business_year,fairness_index
TX,2018,13.42765273282
FL,2018,22.98182143870
IN,2018,24.27080333034
MI,2018,24.48783597682
OH,2018,25.53024790746
PA,2018,26.03513254167
NJ,2018,26.74820004920
GA,2018,31.78649749636
IL,2018,33.77580191405
NC,2018,38.02664552510


Databricks visualization. Run in Databricks to view.

In [0]:
with joined as (
select
p.*,
r.state_code,
FILTER(r.age_category_and_year, x -> x.business_year BETWEEN greatest(p.start_year, 2018) and least(p.end_year, 2024)) AS filtered_structs
from dev_catalog.gold.plans_dim_scd p
join dev_catalog.gold.rates_fact r on p.plan_id = r.plan_id
where p.start_year <= 2024 and p.end_year >= 2018
and r.current_year = 2024
),
exploded as (
SELECT 
    plan_id,
    metal_level,
    new_plan,
    plan_type,
    pandemic_era,
    struct_col.business_year,
    age_category,
    age_rate
FROM joined j 
LATERAL VIEW EXPLODE(filtered_structs) exploded AS struct_col
LATERAL VIEW EXPLODE(
    MAP(
        'age_0_19', struct_col.age_0_19,
        'age_20_29', struct_col.age_20_29,
        'age_30_39', struct_col.age_30_39,
        'age_40_49', struct_col.age_40_49,
        'age_50_59', struct_col.age_50_59,
        'age_60_plus', struct_col.age_60_plus
    )
) exploded_map AS age_category, age_rate
)
SELECT
business_year,
plan_type,
AVG(age_rate) avg_rate
FROM exploded
group by business_year, plan_type

business_year,plan_type,avg_rate
2024,EPO,658.819584
2018,HMO,583.032489
2020,EPO,584.635524
2023,HMO,606.581279
2024,PPO,747.925810
2021,HMO,559.981786
2019,EPO,631.836136
2023,POS,713.507463
2022,EPO,602.131236
2019,HMO,584.363038


Databricks visualization. Run in Databricks to view.

In [0]:
with joined as (
select
p.*,
r.state_code,
FILTER(r.age_category_and_year, x -> x.business_year BETWEEN greatest(p.start_year, 2018) and least(p.end_year, 2024)) AS filtered_structs
from dev_catalog.gold.plans_dim_scd p
join dev_catalog.gold.rates_fact r on p.plan_id = r.plan_id
where p.start_year <= 2024 and p.end_year >= 2018
and r.current_year = 2024
),
exploded as (
SELECT 
    plan_id,
    metal_level,
    new_plan,
    plan_type,
    pandemic_era,
    struct_col.business_year,
    age_category,
    age_rate
FROM joined j 
LATERAL VIEW EXPLODE(filtered_structs) exploded AS struct_col
LATERAL VIEW EXPLODE(
    MAP(
        'age_0_19', struct_col.age_0_19,
        'age_20_29', struct_col.age_20_29,
        'age_30_39', struct_col.age_30_39,
        'age_40_49', struct_col.age_40_49,
        'age_50_59', struct_col.age_50_59,
        'age_60_plus', struct_col.age_60_plus
    )
) exploded_map AS age_category, age_rate
)
SELECT
business_year,
new_plan,
AVG(age_rate) avg_rate
FROM exploded
group by business_year, new_plan

business_year,new_plan,avg_rate
2018,new,592.937916
2020,existing,602.133742
2021,new,570.129211
2020,new,557.269064
2021,existing,587.418135
2018,existing,607.884149
2022,new,576.203383
2019,existing,609.415821
2022,existing,597.774799
2024,existing,658.094076


Databricks visualization. Run in Databricks to view.

In [0]:
with joined as (
select
p.*,
r.state_code,
FILTER(r.age_category_and_year, x -> x.business_year BETWEEN greatest(p.start_year, 2018) and least(p.end_year, 2024)) AS filtered_structs
from dev_catalog.gold.plans_dim_scd p
join dev_catalog.gold.rates_fact r on p.plan_id = r.plan_id
where p.start_year <= 2024 and p.end_year >= 2018
and r.current_year = 2024
),
exploded as (
SELECT 
    plan_id,
    metal_level,
    new_plan,
    plan_type,
    pandemic_era,
    struct_col.business_year,
    age_category,
    age_rate
FROM joined j 
LATERAL VIEW EXPLODE(filtered_structs) exploded AS struct_col
LATERAL VIEW EXPLODE(
    MAP(
        'age_0_19', struct_col.age_0_19,
        'age_20_29', struct_col.age_20_29,
        'age_30_39', struct_col.age_30_39,
        'age_40_49', struct_col.age_40_49,
        'age_50_59', struct_col.age_50_59,
        'age_60_plus', struct_col.age_60_plus
    )
) exploded_map AS age_category, age_rate
)
SELECT
business_year,
pandemic_era,
AVG(age_rate) avg_rate
FROM exploded
group by business_year, pandemic_era

business_year,pandemic_era,avg_rate
2019,pre-covid,615.791901
2020,pre-covid,588.499276
2022,post-covid,588.108155
2024,post-covid,660.417990
2021,post-covid,581.966673
2023,post-covid,628.475402
2018,pre-covid,600.708609


Databricks visualization. Run in Databricks to view.